# Solving a max-flow problem in Pyomo

In [65]:
from pyomo.environ import *

model = AbstractModel()

In [66]:
# Nodes in the network
model.N = Set()
# Network arcs
model.A = Set(within=model.N*model.N)

In [67]:
# Source node
model.s = Param(within=model.N)
# Sink node
model.t = Param(within=model.N)
# Flow capacity limits
model.c = Param(model.A)

In [68]:
# The flow over each arc
model.f = Var(model.A, within=NonNegativeReals)

In [69]:
# Maximize the flow into the sink nodes
def total_rule(model):
    return sum(model.f[i,j] for (i, j) in model.A if j == value(model.t))

model.total = Objective(rule=total_rule, sense=maximize)

In [70]:
# Enforce an upper limit on the flow across each arc
def limit_rule(model, i, j):
    return model.f[i,j] <= model.c[i, j]
model.limit = Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
def flow_rule(model, k):
    if k == value(model.s) or k == value(model.t):
        return Constraint.Skip
    inFlow  = sum(model.f[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.f[i,j] for (i,j) in model.A if i == k)
    return inFlow == outFlow

model.flow = Constraint(model.N, rule=flow_rule)

In [71]:
model

In [72]:
!cat maxflow.dat

set N := Zoo A B C D E Home;
set A := (Zoo,A) (Zoo,B) (A,C) (A,D) (B,A) (B,C) (C,D) (C,E) (D,E) (D,Home) (E,Home);

param s := Zoo;
param t := Home;
param: c :=
Zoo A 11
Zoo B 8
A C 5
A D 8
B A 4
B C 3
C D 2
C E 4
D E 5
D Home 8
E Home 6;


In [75]:
 #This replicates what the pyomo command-line tools does
from pyomo.opt import SolverFactory
opt = SolverFactory("glpk")
instance = model.create_instance(data='maxflow.dat')
results = opt.solve(instance)
#sends results to stdout
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14.0
  Upper bound: 14.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 12
  Number of nonzeros: 30
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.00788903236389
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------

In [76]:
instance.display()

Model unknown

  Variables:
    f : Size=11, Index=A
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
           ('A', 'C') :     0 :   3.0 :  None : False : False : NonNegativeReals
           ('A', 'D') :     0 :   8.0 :  None : False : False : NonNegativeReals
           ('B', 'A') :     0 :   0.0 :  None : False : False : NonNegativeReals
           ('B', 'C') :     0 :   3.0 :  None : False : False : NonNegativeReals
           ('C', 'D') :     0 :   2.0 :  None : False : False : NonNegativeReals
           ('C', 'E') :     0 :   4.0 :  None : False : False : NonNegativeReals
           ('D', 'E') :     0 :   2.0 :  None : False : False : NonNegativeReals
        ('D', 'Home') :     0 :   8.0 :  None : False : False : NonNegativeReals
        ('E', 'Home') :     0 :   6.0 :  None : False : False : NonNegativeReals
         ('Zoo', 'A') :     0 :  11.0 :  None : False : False : NonNegativeReals
         ('Zoo', 'B') :     0 :   3.0 :  None : False : False : No

## we can also solve this using the Ford-Fulkerson algorithms

This below shows a simple implementation of Ford Fulkerson in Python

In [93]:
class Edge(object):
    """Represents an edge in teh graph"""
    def __init__(self, u = 'a', v='b', w=0):
        self.source = u
        self.target = v
        self.capacity = w

    def __repr__(self):
        return "%s->%s:%s" % (self.source, self.target, self.capacity)


class FlowNetwork(object):
    def  __init__(self):
        """Initializes with empty adj and flow dicts"""
        self.adj = {}
        self.flow = {}

    def AddVertex(self, vertex):
        self.adj[vertex] = []

    def GetEdges(self, v):
        return self.adj[v]

    def AddEdge(self, u, v, w = 0):
        if u == v:
            raise ValueError("u == v")
        edge = Edge(u, v, w)
        redge = Edge(v, u, 0)
        edge.redge = redge
        redge.redge = edge
        self.adj[u].append(edge)
        self.adj[v].append(redge)
        # Intialize all flows to zero
        self.flow[edge] = 0
        self.flow[redge] = 0

    def FindPath(self, source, target, path):
        result = None
        if source == target:
            return path
        for edge in self.GetEdges(source):
            residual = edge.capacity - self.flow[edge]
            if residual > 0 and not (edge, residual) in path:
                result = self.FindPath(edge.target, target, path + [(edge, residual)])
            if result != None:
                return result

    def MaxFlow(self, source, target):
        path = self.FindPath(source, target, [])
        print 'path after enter MaxFlow: %s' % path
        for key in self.flow:
            print '    %s:%s' % (key,self.flow[key])
        print '-' * 20
        while path != None:
            flow = min(res for edge, res in path)
            for edge, res in path:
                self.flow[edge] += flow
                self.flow[edge.redge] -= flow
            for key in self.flow:
                print '%s:%s' % (key,self.flow[key])
            path = self.FindPath(source, target, [])
            print 'path inside of while loop: %s' % path
        for key in self.flow:
            print '    %s:%s' % (key,self.flow[key])
        return "Final result:  {0}".format(sum(self.flow[edge] for edge in self.GetEdges(source)))


In [94]:
g = FlowNetwork()
map(g.AddVertex, ['Zoo', 'A', 'B', 'C', 'D', 'E','Home'])
g.AddEdge('Zoo', 'A', 11)
g.AddEdge('Zoo', 'B', 8)
g.AddEdge('A', 'C', 5)
g.AddEdge('A', 'D', 8)
g.AddEdge('B', 'A', 4)
g.AddEdge('B', 'C', 3)
g.AddEdge('C', 'D', 2)
g.AddEdge('C', 'E', 4)
g.AddEdge('D', 'E', 5)
g.AddEdge('D', 'Home', 8)
g.AddEdge('E', 'Home', 6)
print g.MaxFlow('Zoo', 'Home')

path after enter MaxFlow: [(Zoo->A:11, 11), (A->C:5, 5), (C->D:2, 2), (D->E:5, 5), (E->Home:6, 6)]
    B->Zoo:0:0
    Zoo->A:11:0
    Zoo->B:8:0
    B->C:3:0
    A->Zoo:0:0
    C->A:0:0
    A->C:5:0
    D->A:0:0
    A->D:8:0
    A->B:0:0
    B->A:4:0
    C->B:0:0
    D->C:0:0
    C->D:2:0
    E->C:0:0
    D->E:5:0
    C->E:4:0
    E->D:0:0
    Home->D:0:0
    D->Home:8:0
    Home->E:0:0
    E->Home:6:0
--------------------
B->Zoo:0:0
Zoo->A:11:2
Zoo->B:8:0
B->C:3:0
A->Zoo:0:-2
C->A:0:-2
A->C:5:2
D->A:0:0
A->D:8:0
A->B:0:0
B->A:4:0
C->B:0:0
D->C:0:-2
C->D:2:2
E->C:0:0
D->E:5:2
C->E:4:0
E->D:0:-2
Home->D:0:0
D->Home:8:0
Home->E:0:-2
E->Home:6:2
path inside of while loop: [(Zoo->A:11, 9), (A->Zoo:0, 2), (Zoo->B:8, 8), (B->A:4, 4), (A->C:5, 3), (C->A:0, 2), (A->D:8, 8), (D->C:0, 2), (C->E:4, 4), (E->D:0, 2), (D->E:5, 3), (E->Home:6, 4)]
B->Zoo:0:-2
Zoo->A:11:2
Zoo->B:8:2
B->C:3:0
A->Zoo:0:-2
C->A:0:-2
A->C:5:2
D->A:0:-2
A->D:8:2
A->B:0:-2
B->A:4:2
C->B:0:0
D->C:0:0
C->D:2:0
E->C:0:-2
D->E: